In [2]:
import pandas as pd
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from random import randint
import nltk.data
import re
import enchant
from enchant.checker import SpellChecker
path ='./'
dataset = pd.read_csv(path + 'train.csv'.format(1), engine='python')
dataset.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
word = 'toxic'
print(dataset[word][dataset[word] == 0].value_counts())
print(dataset[word][dataset[word] == 1].value_counts())

0    144277
Name: toxic, dtype: int64
1    15294
Name: toxic, dtype: int64


In [8]:
!pip install joblib textblob -U

Requirement already up-to-date: nltk>=3.1 in c:\python27\lib\site-packages (from textblob)
Requirement already up-to-date: six in c:\python27\lib\site-packages (from nltk>=3.1->textblob)


You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [41]:
from joblib import Parallel, delayed
from textblob import TextBlob
from textblob.translate import NotTranslated

import argparse
import os
import pandas as pd

In [71]:
ds=dataset[dataset['toxic']==1]
ds=ds[['comment_text','toxic']][0:100]

In [72]:
tokenizer = TreebankWordTokenizer()
chkr = SpellChecker("en_UK", "en_US")

def low(x):
    # spelling checker
    x = re.sub(r'[\W_]+', ' ', x)
    chkr.set_text(x)
    for err in chkr:
        if len(err.suggest()) > 0:
            sug = err.suggest()[0]
            err.replace(sug)
            x = chkr.get_text()
    
    return x.lower()


ds['comment_text'] = ds['comment_text'].apply(lambda x: low(x))


In [73]:
NAN_WORD = "_NAN_"


def translate(comment, language):
    if hasattr(comment, "decode"):
        comment = comment.decode("utf-8")

    text = TextBlob(comment)
    try:
        text = text.translate(to='de')
        text = text.translate(to=language)
        text = text.translate(to="en")
    except NotTranslated:
        pass

    return str(text)

In [74]:
dp=ds[["comment_text", "toxic"]][0:70]
df =ds[["comment_text", "toxic"]][0:70].copy()

def upsample(comment, label):
    global dp
    l = ['es', 'fr', 'it']
    for language in l:
        new_comment = translate(comment, language)
        dp=dp.append(pd.Series({"comment_text":new_comment,"toxic":label}),ignore_index=True)
        
df.apply(lambda x: upsample(x["comment_text"], x["toxic"]), axis=1)

6      None
12     None
16     None
42     None
43     None
44     None
51     None
55     None
56     None
58     None
59     None
65     None
79     None
86     None
105    None
151    None
159    None
168    None
176    None
181    None
201    None
206    None
211    None
218    None
231    None
238    None
268    None
278    None
286    None
295    None
       ... 
415    None
423    None
429    None
437    None
439    None
442    None
451    None
476    None
497    None
503    None
519    None
520    None
521    None
529    None
579    None
582    None
590    None
600    None
602    None
604    None
608    None
610    None
632    None
638    None
641    None
642    None
643    None
655    None
665    None
679    None
Length: 70, dtype: object

In [20]:
#just one time
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import word_tokenize, sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
import enchant
from enchant.checker import SpellChecker

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gatto\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gatto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [139]:
trial=ds[0:2]


In [140]:

# Load a text file if required


# Load the pretrained neural net
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


for text in trial["comment_text"]:
    output=""
    # Tokenize the text
    tokenized = tokenizer.tokenize(text)

    # Get the list of words from the entire text
    words = word_tokenize(text)

    # Identify the parts of speech
    tagged = nltk.pos_tag(words)

    for i in range(0,len(words)):
        
        replacements = []

        # Only replace nouns with nouns, vowels with vowels etc.
        for syn in wordnet.synsets(words[i]):

            # Do not attempt to replace proper nouns or determiners
            if tagged[i][1] == 'NNP' or tagged[i][1] == 'DT':
                break

            # The tokenizer returns strings like NNP, VBP etc
            # but the wordnet synonyms has tags like .n.
            # So we extract the first character from NNP ie n
            # then we check if the dictionary word has a .n. or not 
            word_type = tagged[i][1][0].lower()
            if syn.name().find("."+word_type+"."):
                # extract the word only
                r = syn.name()[0:syn.name().find(".")]
                replacements.append(r)

        if len(replacements) > 0:
            # Choose a random replacement
            replacement = replacements[randint(0,len(replacements)-1)]
            output = output + " " + replacement
        else:
            # If no replacement could be found, then just use the
            # original word
            output = output + " " + words[i]
    print output
    trial=trial.append(pd.Series({"comment_text":output,"toxic":1}),ignore_index=True)

 cocksucker ahead you peeing about on my work
 hey what be information_technology talk what cost information_technology an exclusive group of some p taliban world_health_organization be beneficial astatine destroy self appointed purist world_health_organization gang up any one world_health_organization ask them doubt bat their anti social and destructive not contribution at p ask sinuosity to clean astir his behavior than issue maine absurd warn


In [141]:
trial

,comment_text,toxic
0,cocksucker before you piss around on my work,1
1,hey what is it talk what is it an exclusive gr...,1
2,cocksucker ahead you peeing about on my work,1
3,hey what be information_technology talk what ...,1


In [4]:

def tok_and_rem(x):
    s = x.split()
    l = list(s)
    for word in s:
        if (len(word) <= 2):
            l.remove(word)
        elif word in stopwords.words('english'):
            l.remove(word)
    return l

dp['comment_text'] = dp['comment_text'].apply(lambda x: tok_and_rem(x))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gatto\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True